In [25]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nib
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform
import subprocess

In [26]:
data_dir = Path("/Users/shufanzhang/disks/rom_Sh/2024/visual/AOT/derivatives")

subj_nr = 1
sub = 'sub-{0:02d}'.format(subj_nr)
cx_sub = "EO_" + sub

fs_dir = data_dir/ "fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer"
fs_subj_dir = Path(fs_dir, sub)

glm_dir = data_dir / 'glmsingle'     
#glm_subj_dir = glm_dir / sub
#glm_subj_gfeat_dir = glm_subj_dir / 'func' / 'sub-01_task-ER_space-T1w_desc-preproc_bold.gfeat'

ctx_dir = data_dir / 'cortex' / 'outputs'

In [27]:
print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

cortex.database.default_filestore: /Users/shufanzhang/opt/anaconda3/share/pycortex/db
cortex.options.usercfg: /Users/shufanzhang/Library/Application Support/pycortex/options.cfg


In [28]:
cortex.freesurfer.import_subj(fs_subject=sub,
                                cx_subject=cx_sub,
                                freesurfer_subject_dir=fs_dir)

error: mghRead(/Users/shufanzhang/disks/rom_Sh/2024/visual/AOT/derivatives/fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer/sub-01/mri/T1.mgz, -1): could not open file 


CalledProcessError: Command '['mri_convert', '/Users/shufanzhang/disks/rom_Sh/2024/visual/AOT/derivatives/fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer/sub-01/mri/T1.mgz', '/Users/shufanzhang/opt/anaconda3/share/pycortex/db/EO_sub-01/anatomicals/raw.nii.gz']' returned non-zero exit status 1.

In [ ]:
example_R2_fn = "/Users/shufanzhang/disks/rom_Sh/2024/visual/AOT/derivatives/glmsingle/mainexp/sub-001_ses-01_T1W_nordicstc_mainfull/TYPED_FITHRF_GLMDENOISE_RR/R2.nii"
bref = nib.load(example_R2_fn)
transform = cortex.xfm.Transform(np.identity(4), bref)
transform.save(cx_sub, 'fmriprep', 'magnet')

In [ ]:
example_anat_fn = fs_subj_dir / 'mri' / 'T1.mgz'
aref = nib.load(example_anat_fn)
transform = cortex.xfm.Transform(np.identity(4), aref)
transform.save(cx_sub, 'anat', 'magnet')

In [ ]:
mean_func_vcx = cortex.Volume(str(example_R2_fn), cx_sub, 'fmriprep', cmap='bone', vmin=0, vmax=16000)
T1_anat_vcx = cortex.Volume(str(fs_subj_dir / 'mri' / 'T1.mgz'), cx_sub, 'anat', cmap='Greens', vmin=0, vmax=1)

ds = {"mean_func": mean_func_vcx, "myelin": T1_anat_vcx}
for cope, name in zip((1,2,3,4), ["positive", "negative", "pos-neg", "neg-pos"]):
    fracvalue = "/Users/shufanzhang/disks/rom_Sh/2024/visual/AOT/derivatives/glmsingle/mainexp/sub-001_ses-01_T1W_nordicstc_mainfull/TYPED_FITHRF_GLMDENOISE_RR/FRACvalue.nii"
    cope_vcx = cortex.Volume(str(fracvalue), cx_sub, 'fmriprep', cmap='coolwarm', vmin=-4.5, vmax=4.5)
    ds[name] = cope_vcx

cortex.webgl.show(ds)
cortex.webgl.make_static(ctx_dir / sub, ds)

Started server on port 43159


Stopping server
